# Binning bivariate

Discretizes the data into a regular grid (computes a binned approximation) using the nearest binning technique.

Statistical data binning is a way to group a number of more or less continuous values into a smaller number of "bins". For example, if you have irregularly distributed data over the oceans, you can organize these observations into a smaller number of geographical intervals (for example, by grouping them all five degrees into latitudes and longitudes).

In [ ]:
import os
import numpy as np
import xarray as xr
import pyinterp.binning

In this example, we will calculate the statistics on the MSS grid with boxes of 5 degrees in latitudes and longitudes.

In [ ]:
binned = pyinterp.binning.NearestBivariate(
    pyinterp.Axis(np.arange(-180, 180, 5), is_circle=True),
    pyinterp.Axis(np.arange(-90, 95, 5)))

In [ ]:
binned

Load our test sets into memory.

In [ ]:
ds = xr.load_dataset("../tests/dataset/mss.nc")

In [ ]:
lon, lat = np.meshgrid(ds.lon, ds.lat)

We push the loaded data into the different defined bins.

In [ ]:
binned.push(lon, lat, ds.mss)

We visualize our result

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

fig = plt.figure(figsize=(9, 4.5))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
lon, lat = np.meshgrid(binned.x, binned.y, indexing='ij')
ax.pcolormesh(lon, lat, binned.variable('mean'), cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Mean MSS per 5 degree bin.")